# Algorithms

## 3.17 Advantage Actor-Critic (A2C)

### Baseline Invariance

策略梯度有一个重要的性质：**baseline invariance**。这意味着如果我们给奖励函数加上一个常数，最优策略不会改变，即：
$$
\mathbb{E}_{S \sim \eta, A \sim \pi}[\nabla_{\theta} \log \pi (A|S, \theta_t)q_{\pi}(S, A)] = \mathbb{E}_{S \sim \eta, A \sim \pi}[\nabla_{\theta} \log \pi (A|S, \theta_t)q_{\pi}(S, A) + b(S)]
$$
其中，$b(S)$ 是一个与状态 $S$ 相关的标量函数。

### Advantage Function 

一个简洁的baseline 是动作价值函数的期望——状态价值：
$$
b(s) = \mathbb{E}_{A \sim \pi}[q_{\pi}(s, A)]=v_{\pi}(s), \ \ \ \forall s \in \mathcal{S}
$$

当$b(s)=v_{\pi}(s)$时，策略梯度算法中的参数更新变为：
$$
\begin{aligned}
\theta_{t+1} &= \theta_t + \alpha \mathbb{E} \Big[\nabla_{\theta_t} \log \pi (A|S, \theta_t) [q_{\pi}(S, A) - v_{\pi}(S)] \Big] \\
&= \theta_t + \alpha \mathbb{E} \Big[\nabla_{\theta_t} \log \pi (A|S, \theta_t) A^{\pi}(S) \Big]
\end{aligned}
$$

其中，$A^{\pi}(S)$ 是优势函数（Advantage Function）：
$$
A^{\pi}(s, a) = q_{\pi}(S, A) - v_{\pi}(S)
$$

表示一个动作相对于当前状态下其他动作的相对优势。

随机梯度近似（Stochastic Gradient Descent）用于最小化损失函数，通过以下公式更新参数：
$$
\begin{aligned}
\theta_{t+1} &= \theta_t + \alpha \nabla_{\theta} \log \pi (a_t|s_t; \theta_t)[q_{\pi}(s_t, a_t) - v_{\pi}(s_t)] \\
&= \theta_t + \alpha \nabla_{\theta} \log \pi (a_t|s_t; \theta_t) A^{\pi}(s_t, a_t)
\end{aligned}
$$

策略基于$q_{\pi}(s_t, a_t)$和$v_{\pi}(s_t)$之间的差异进行更新————直观上理解是算法试图在状态$s$下选择一个相对于其他动作具有更大价值的动作。

- 如果$A^{\pi}(s_t, a_t) > 0$，则策略会倾向于增加该动作的概率；如果$A^{\pi}(s_t, a_t) < 0$，则策略会倾向于减少该动作的概率。

- 如果$q_{\pi}(s_t, a_t)$和$v_{\pi}(s_t)$被Monte Carlo学习估计，则称为REINFORCE Witho Baseline算法；如果使用时序差分（TD）学习方法估计，则称为Advantage Actor-Critic算法。


### Advantage Actor-Critic Algorithm (A2C)

- 初始化Actor策略网络$\pi(a|s; \theta)$和Critic价值网络$v(s; w)$的参数$\theta$和$w$、初始化学习率$\alpha_{\theta}$、$\alpha_{w}$、折扣因子$\gamma$
- $for \ e \rightarrow E \ do:$
  - 遵循当前策略$\pi(a|s_t; \theta_t)$生成动作$a_t$并与环境交互，观测奖励$r_{t+1}$和下个状态$s_{t+1}$
  - Advantage(TD error):
    - $A(s_t, a_t) \leftarrow r_{t+1} + \gamma v(s_{t+1}; w) - v(s_t; w)$
  - Policy Update (Actor):
    - $\theta_{t+1} \leftarrow \theta_t + \alpha_{\theta} A(s_t, a_t) \nabla_{\theta} log\pi(a_t|s_t; \theta_t)$
  - Value Update (Critic):
    - $w_{t+1} \leftarrow w_t + \alpha_w A(s_t, a_t) \nabla_w v(s_t; w_t)$

### Example